Acknowlegements: Thanks to Verena (and Maxim) for much of this lovely code!

In [0]:
import urllib
import pandas as pd
import numpy as np

In [0]:
def get_comments(filename, url=True):
    if url:
        comments = []
        with urllib.request.urlopen(filename) as f:
            for line in f:
                if line.startswith(b'#'):
                    comments.append(line.decode("utf-8"))
                else:
                    break
        return comments
    with open(filename, 'r', encoding='utf8') as f:
        commentiter = takewhile(lambda s: s.startswith('#'), f)
        comments = list(commentiter)
    return comments

In [0]:
TRAIN_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AFDEFD4F7VOZTUSCLUDLSUC6LEXMC'
DEV_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/dev-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AFDEFD3UUERAVPLXIMU52MS6LEXSI'
DEV_LABELS_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train%2Bdev-improved.tsv?token=AFDEFD2UDJVAC5JW2NOOPZK6LKNCS'

In [48]:
comments = get_comments(TRAIN_URL)
train_df = pd.read_csv(TRAIN_URL, sep='\t', skiprows=len(comments), quoting=3)
train_input = train_df.groupby('sent_id')['token'].apply(list).to_frame()
print(train_df.head())
print(train_input.head())
print(list(train_df.columns))
print(train_df.shape[0])

   document_id  sent_id  token_start  token_end  ... PUNCT SYM  VERB  X
0    111111111        1            0          4  ...     0   0     0  0
1    111111111        1            5         11  ...     0   0     0  0
2    111111111        1           12         20  ...     0   0     0  0
3    111111111        1           21         23  ...     0   0     0  0
4    111111111        1           24         34  ...     0   0     0  0

[5 rows x 24 columns]
                                                     token
sent_id                                                   
1        [Next, plague, outbreak, in, Madagascar, could...
2        [Geneva, -, The, World, Health, Organisation, ...
3        [", The, next, transmission, could, be, more, ...
4        [An, outbreak, of, both, bubonic, plague, ,, w...
5        [has, killed, more, than, 200, people, in, the...
['document_id', 'sent_id', 'token_start', 'token_end', 'token', 'label', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 

In [5]:
#delete 730081389
comments = get_comments(DEV_URL)
dev_df = pd.read_csv(DEV_URL, sep='\t', skiprows=len(comments), quoting=3)
dev_input = dev_df.groupby('sent_id')['token'].apply(list).to_frame()
print(dev_df.head())
print(dev_input.head())
print(list(dev_df.columns))

   document_id  sent_id  token_start  token_end  ... PUNCT  SYM  VERB  X
0    730081389        1            0          6  ...     0    0     0  0
1    730081389        1            7         10  ...     0    0     1  0
2    730081389        1           11         21  ...     0    0     0  0
3    730081389        1           22         26  ...     0    0     1  0
4    730081389        1           27         29  ...     0    0     0  0

[5 rows x 23 columns]
                                                     token
sent_id                                                   
1        [Police, had, previously, gone, to, home, wher...
2        [CLEVELAND, —, Police, invstigating, domestic,...
3        [police, reports, from, the, Columbus, suburb,...
4        [Westerville, Officers, Eric, Joering, ,, 39, ...
5        [The, suspect, ,, 30-year, -, old, Quentin, Sm...
['document_id', 'sent_id', 'token_start', 'token_end', 'token', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 'CC

In [49]:
dev_df = dev_df[dev_df.document_id != 730081389]
print(dev_df.head())
print(dev_df.shape[0])

     document_id  sent_id  token_start  token_end  ... PUNCT  SYM  VERB  X
347    730093263       22            0          7  ...     0    0     0  0
348    730093263       22            7          9  ...     0    0     0  0
349    730093263       22           10         21  ...     0    0     0  0
350    730093263       22           22         27  ...     0    0     0  0
351    730093263       22           27         28  ...     1    0     0  0

[5 rows x 23 columns]
66826


In [15]:
dev_labels_df = pd.read_csv(DEV_LABELS_URL, sep='\t', skiprows=1, quoting=3)
print(dev_labels_df.head())

   document_id  sent_id  token_start  token_end        token label
0    730093263        1            0          7      America     O
1    730093263        1            7          9           's     O
2    730093263        1           10         21  Immigration     O
3    730093263        1           22         27        Voice     O
4    730093263        1           27         28            .     O


In [34]:
result_df = dev_df.merge(dev_labels_df, how='inner', left_on=['document_id', 'token_start'], right_on=['document_id', 'token_start'])
print(result_df.head())
print(list(result_df.columns))

   document_id  sent_id_x  token_start  ...  token_end_y      token_y  label
0    730093263         22            0  ...            7      America      O
1    730093263         22            7  ...            9           's      O
2    730093263         22           10  ...           21  Immigration      O
3    730093263         22           22  ...           27        Voice      O
4    730093263         22           27  ...           28            .      O

[5 rows x 27 columns]
['document_id', 'sent_id_x', 'token_start', 'token_end_x', 'token_x', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'X', 'sent_id_y', 'token_end_y', 'token_y', 'label']


In [0]:
result_df = result_df.drop(columns=['sent_id_y', 'token_end_y', 'token_y'])

In [40]:
result_df = result_df.rename(columns={"sent_id_x": "sent_id", "token_end_x": "token_end", "token_x":"token"})
print(list(result_df.columns))

['document_id', 'sent_id', 'token_start', 'token_end', 'token', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'X', 'label']


In [0]:
result_df_1 = result_df[['document_id', 'sent_id', 'token_start', 'token_end', 'token', 'label', 'positive', 'negative', 'arglex_any', 'ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SYM', 'VERB', 'X']]

In [47]:
result_df_1.head()

,document_id,sent_id,token_start,token_end,token,label,positive,negative,arglex_any,ADJ,ADP,ADV,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SYM,VERB,X
0,730093263,22,0,7,America,O,0.000000,0.000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,730093263,22,7,9,'s,O,0.000000,0.000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,730093263,22,10,21,Immigration,O,0.000000,0.125,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,730093263,22,22,27,Voice,O,0.019231,0.000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,730093263,22,27,28,.,O,0.000000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [45]:
frames = [train_df, result_df_1]
train_dev_df = pd.concat(frames)
train_dev_df.shape[0]

468112

In [0]:
file = train_dev_df.to_csv("train_dev_set.tsv", sep='\t')

In [0]:
from google.colab import files
files.download("train_dev_set.tsv") 